In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import haversine as hs
import geopandas as gpd
import osmnx as ox

In [2]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

In [3]:
for index, row in df_grid.iterrows():
    df_grid.loc[index, 'substation_1_name'] = df_substations.loc[row.substation_1, 'name']
    df_grid.loc[index, 'substation_2_name'] = df_substations.loc[row.substation_2, 'name']
    loc1=(df_substations.loc[row.substation_1, 'lat'], df_substations.loc[row.substation_1, 'lon'])
    loc2=(df_substations.loc[row.substation_2, 'lat'], df_substations.loc[row.substation_2, 'lon'])
    df_grid.loc[index, 'calculated_length'] = hs.haversine(loc1,loc2)

In [6]:
df_substations_fr = df_substations[(df_substations.zone == 'FR')]
df_substations_rte = pd.read_excel('./postes-electriques-rte.xlsx')
display(df_substations_rte)

,Code poste,Nom poste,Etat,Tension (kV),Longitude poste (DD),Latitude poste (DD),Géo-point poste (DD),FONCTION DU POSTE
0,S.GUY,SAUVETERRE-DE-GUYENNE,EN EXPLOITATION,225kV,-0.086927,44.713869,"44.71386904155741, -0.08692681055536502",Poste de transformation
1,ZCAI6,PIQUAGE A CAIRAC,EN EXPLOITATION,63kV,2.226454,42.973795,"42.97379484396188, 2.2264543987320597",Poste de piquage
2,F.FRE,FAUX-FRESNAY,EN EXPLOITATION,400kV,3.976210,48.649824,"48.64982394686412, 3.9762098808798574",Poste de transformation
3,ZG.HA,PIQUAGE A GERBECOURT ET HAPLEMONT,EN EXPLOITATION,225kV,6.135854,48.485322,"48.48532182451375, 6.1358539678466455",Poste de piquage
4,1ALBI,ALBIES (TALC DE LUZENAC) /LUZENAC 20KV,EN EXPLOITATION,<45kV,1.698009,42.778237,"42.778236815730004, 1.6980094921441378",Poste de transformation
...,...,...,...,...,...,...,...,...
4998,ZAMIG,PIQUAGE A AMIGNE,EN EXPLOITATION,225kV,0.291467,47.991498,"47.991497765051, 0.29146660858234325",Poste de piquage
4999,SSSA6,SAINT-SAVOURNIN,EN EXPLOITATION,63kV,5.548211,43.413629,"43.41362910404203, 5.548211248576792",Poste de transformation
5000,C.JAN,CAP JANET,EN EXPLOITATION,225kV,5.352869,43.336043,"43.33604281751801, 5.352868759377503",Poste de transformation
5001,V.ILE,VENT-DES-ILES,ACCORD ADMINISTRATIF,225kV,-2.154767,46.885261,"46.885261194830264, -2.154767359712274",Poste de transformation


In [11]:
for index, row in df_substations_fr.iterrows():
    if row['name'] in df_substations_rte['Nom poste'].tolist():
        df_substations_fr.loc[index, 'lat_fr'] = df_substations_rte[df_substations_rte['Nom poste'] == row['name']].iloc[0]['Latitude poste (DD)']
        df_substations_fr.loc[index, 'lon_fr'] = df_substations_rte[df_substations_rte['Nom poste'] == row['name']].iloc[0]['Longitude poste (DD)']
    
display(df_substations_fr)

C:\Users\balazs.riskutia\AppData\Local\Temp\ipykernel_17556\205393515.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_substations_fr.loc[index, 'lon_fr'] = df_substations_rte[df_substations_rte['Nom poste'] == row['name']].iloc[0]['Longitude poste (DD)']


,name,zone,tso,lat,lon,lat_fr,lon_fr
1,TUILIERES,FR,RTE,44.846132,0.633183,44.844456,0.635854
3,MITRY-MORY (E.D.F.),FR,RTE,48.966663,2.631343,48.967130,2.631444
5,VERTOU,FR,RTE,47.181839,-1.482085,47.182927,-1.481918
10,BARNABOS,FR,RTE,49.657112,1.012345,49.657738,1.012524
11,SACLAY (PLATEAU),FR,RTE,48.720369,2.200296,48.720370,2.200294
...,...,...,...,...,...,...,...
1468,MONTOIS,FR,RTE,49.212973,6.011512,49.213230,6.010978
1477,ETUPES,FR,RTE,47.507393,6.847866,47.507469,6.848120
1479,BUIRE,FR,RTE,49.915886,4.049269,49.916118,4.049075
1490,CHAMBRY,FR,RTE,48.984105,2.907271,48.984759,2.907483


In [13]:
for index, row in df_substations_fr.iterrows():
        i = df_substations.index[df_substations.name == row['name']].tolist()[0]
        df_substations.loc[i, 'lat'] = row['lat_fr']
        df_substations.loc[i, 'lon'] = row['lon_fr']

In [16]:
df_substations.to_excel("substations.xlsx")